In [29]:
import sqlite3
from sqlite3 import Error

In [36]:
def sql_connection(dbase):
    try:
        con = sqlite3.connect(dbase)
        return con
    except Error:
        print(Error)
        
   # finally:
   #     con.close()            

In [61]:
def sql_table(con, create_sql):
    cursorObj = con.cursor()
    cursorObj.execute(create_sql)
    con.commit()  

In [37]:
con = sql_connection('FlightDatabase.db')

In [133]:
create_sql ="""
CREATE TABLE FlightLeg2(
id INTEGER PRIMARY KEY AUTOINCREMENT, 
tailNumber  CHAR(50), 
sourceAirportCode CHAR(3), 
destinationAirportCode CHAR(3), 
sourceCountryCode CHAR(3), 
destinationCountryCode CHAR(3),
departureTimeUtc DATETIME,
landingTimeUtc DATETIME
)
"""

In [62]:
create_sql="""
CREATE TABLE test(
id integer PRIMARY KEY, 
tailNumber  text
)
"""

In [134]:
sql_table(con,create_sql )

In [80]:
def sql_insert(con, insert_sql):
    cursorObj = con.cursor()    
    cursorObj.execute(insert_sql)
    con.commit()

In [119]:
entities = (2, '2', 3, '4', '5', '6', '7', '8')
insert_sql = f"""INSERT INTO FlightLeg VALUES {entities}"""

In [ ]:
entities = (2, '2', 3, '4', '5', '6', '7', '8')
insert_sql = f"""INSERT INTO FlightLeg VALUES {entities}"""

In [103]:
entities = (2, 'text2')
insert_sql = f"""
INSERT INTO test VALUES{entities}
"""

In [115]:
insert_sql

"INSERT INTO FlightLeg VALUES (1, '2', 3, '4', '5', '6', '7', '8')"

In [120]:
sql_insert(con, insert_sql)

In [99]:
def sql_fetch(con, select_sql):
    cursorObj = con.cursor()
    cursorObj.execute(select_sql)
    rows = cursorObj.fetchall()
    for row in rows:
        print(row)

In [149]:
select_sql= """
SELECT name 
from sqlite_master 
where type= "table";
"""

In [101]:
select_sql= """
SELECT * 
FROM test 
WHERE 1=1;
"""

In [186]:
select_sql= """
SELECT * 
FROM FlightLeg
WHERE 1=1;
"""
print(sql_fetch(con, select_sql))

In [187]:
print(sql_fetch(con, select_sql))

(1, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00')
None


In [125]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 4665-F0B9

 Directory of C:\Users\PiT\PycharmProjects\ssn\Zadanie_2

09.01.2022  13:14    <DIR>          .
09.01.2022  13:14    <DIR>          ..
09.01.2022  12:30    <DIR>          .ipynb_checkpoints
09.01.2022  13:14             5˙580 BitPeak_JDE_Z2_PIOTR_STANI.ipynb
09.01.2022  13:12            12˙288 FlightDatabase.db
09.01.2022  13:13            63˙132 flightlegs.csv
               3 File(s)         81˙000 bytes
               3 Dir(s)  239˙297˙527˙808 bytes free


In [121]:
import pandas as pd

In [129]:
df = pd.read_csv('flightlegs.csv', delimiter=';')
df

,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time
0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00
1,A6-ENE,ACC,GHA,ABJ,CIV,2021-11-29 13:35:00,2021-11-29 14:17:00
2,A6-END,DXB,ARE,MNL,PHL,2021-11-27 23:50:11,2021-11-28 07:58:00
3,A6-EPD,IST,TUR,DXB,ARE,2021-11-28 17:51:53,2021-11-28 21:31:00
4,A6-EUU,JED,SAU,DXB,ARE,2021-11-28 08:53:00,2021-11-28 11:10:00
...,...,...,...,...,...,...,...
995,A6-EOV,DXB,ARE,DME,RUS,2021-11-28 13:01:00,2021-11-28 17:48:00
996,A6-EGW,ZAZ,ESP,DXB,ARE,2021-11-28 17:37:30,2021-11-29 00:08:00
997,A6-EGT,BKK,THA,DXB,ARE,2021-11-28 13:22:06,2021-11-28 19:33:00
998,A6-EGP,DXB,ARE,LCK,USA,2021-11-29 04:51:00,2021-11-29 19:07:00


In [144]:
df[0:1]

,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time
0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00


In [188]:
row = df.loc[1, :].values.tolist()  
str(row)[1:-1]

"'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00'"

In [190]:
entities = f"{str(row)[1:-1]}"
insert_sql = f"""
INSERT INTO FlightLeg2 VALUES(NULL,{entities})
"""
insert_sql

"\nINSERT INTO FlightLeg2 VALUES(NULL,'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00')\n"

In [192]:
sql_insert(con, insert_sql)

In [197]:
select_sql= """
SELECT * 
FROM FlightLeg2
WHERE 1=1;
"""
print(sql_fetch(con, select_sql))

(1, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00')
(2, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00')
(3, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00')
(4, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00')
(5, 'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00')
(6, 'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00')
(7, 'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00')
(8, 'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00')
(9, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00')
(10, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00')
(11, 'A6-EUM', 'AMS', 'NLD', 'DXB', 'ARE', '2021-11-29 14:06:00', '2021-11-29 19:51:00')
(12, 'A6-EUD', 'DXB', 'ARE', '

In [194]:
print(len(df.index))

1000


In [196]:
con = sql_connection('FlightDatabase.db')
for x in range(0,len(df.index)):
    row = df.loc[x, :].values.tolist()
    
    entities = f"{str(row)[1:-1]}"
    insert_sql = f"""
    INSERT INTO FlightLeg2 VALUES(NULL,{entities})
    """
    print(insert_sql)
    
    sql_insert(con, insert_sql)
    


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:3


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECX', 'BNE', 'AUS', 'DXB', 'ARE', '2021-11-30 11:11:30', '2021-12-01 00:59:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENJ', 'DAC', 'BGD', 'DXB', 'ARE', '2021-11-28 04:59:00', '2021-11-28 09:46:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGE', 'DXB', 'ARE', 'GRU', 'BRA', '2021-11-30 17:07:12', '2021-12-01 07:27:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUT', 'JFK', 'USA', 'DXB', 'ARE', '2021-11-29 16:12:00', '2021-11-30 04:03:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPE', 'DXB', 'ARE', 'ISB', 'PAK', '2021-11-29 23:20:00', '2021-11-30 02:00:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENK', 'BLQ', 'ITA', 'DXB', 'ARE', '2021-11-28 13:49:00', '2021-11-28 19:10:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENU', 'DXB', 'ARE', 'DEL', 'IND', '2021-11-30 18:08:41', '2021-11-30 20:53:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECH', 'DSS', 'SEN', 'DXB', 'ARE', '2021-11-28 17:57:1


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPB', 'DXB', 'ARE', 'BOM', 'IND', '2021-11-29 17:47:40', '2021-11-29 20:28:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQD', 'PER', 'AUS', 'DXB', 'ARE', '2021-11-30 14:16:56', '2021-12-01 00:47:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENH', 'CPH', 'DNK', 'DXB', 'ARE', '2021-11-28 13:23:10', '2021-11-28 19:19:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPQ', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-28 23:42:14', '2021-11-29 07:50:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUS', 'MRU', 'MUS', 'DXB', 'ARE', '2021-11-29 19:14:00', '2021-11-30 01:07:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUM', 'DXB', 'ARE', 'AMS', 'NLD', '2021-11-29 04:39:00', '2021-11-29 11:46:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGQ', 'ACC', 'GHA', 'DXB', 'ARE', '2021-11-28 18:55:00', '2021-11-29 02:21:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUS', 'DXB', 'ARE', 'MRU', 'MUS', '2021-11-28 22:44:0


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENO', 'DXB', 'ARE', 'ISB', 'PAK', '2021-11-29 18:07:53', '2021-11-29 20:34:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECO', 'DXB', 'ARE', 'BLR', 'IND', '2021-11-30 17:51:00', '2021-11-30 21:32:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGL', 'DXB', 'ARE', 'NBO', 'KEN', '2021-11-30 12:22:00', '2021-11-30 17:05:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EOV', 'DXB', 'ARE', 'VIE', 'AUT', '2021-11-29 05:21:00', '2021-11-29 11:23:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EVC', 'MRU', 'MUS', 'DXB', 'ARE', '2021-11-30 19:33:00', '2021-12-01 01:45:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EVK', 'IAD', 'USA', 'DXB', 'ARE', '2021-11-29 16:13:00', '2021-11-30 04:17:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENE', 'CMB', 'LKA', 'MLE', 'MDV', '2021-11-28 14:47:05', '2021-11-28 15:58:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQC', 'DXB', 'ARE', 'LYS', 'FRA', '2021-11-28 11:16:0


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQJ', 'ATH', 'GRC', 'DXB', 'ARE', '2021-11-29 15:52:00', '2021-11-29 20:06:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EVD', 'AMM', 'JOR', 'DXB', 'ARE', '2021-11-29 15:59:00', '2021-11-29 18:18:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECI', 'KUL', 'MYS', 'AKL', 'NZL', '2021-11-30 13:08:00', '2021-11-30 22:38:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGC', 'DAR', 'TZA', 'DXB', 'ARE', '2021-11-30 13:19:08', '2021-11-30 18:16:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EEA', 'MAD', 'ESP', 'DXB', 'ARE', '2021-11-27 20:59:00', '2021-11-28 03:08:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUC', 'DXB', 'ARE', 'MUC', 'DEU', '2021-11-28 05:28:00', '2021-11-28 11:45:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGE', 'DXB', 'ARE', 'COK', 'IND', '2021-11-28 23:02:00', '2021-11-29 02:29:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPE', 'DXB', 'ARE', 'CGK', 'IDN', '2021-11-28 01:07:0

...


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQJ', 'SIN', 'SGP', 'DXB', 'ARE', '2021-11-30 17:01:00', '2021-11-30 23:53:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPZ', 'JFK', 'USA', 'MXP', 'ITA', '2021-11-30 04:35:53', '2021-11-30 11:37:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGS', 'BKK', 'THA', 'HKT', 'THA', '2021-11-29 14:25:00', '2021-11-29 15:28:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EBR', 'CKY', 'GIN', 'DSS', 'SEN', '2021-11-30 15:38:59', '2021-11-30 16:39:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EEN', 'MAD', 'ESP', 'DXB', 'ARE', '2021-11-28 13:57:00', '2021-11-28 20:16:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECJ', 'DXB', 'ARE', 'MEL', 'AUS', '2021-11-30 23:37:00', '2021-12-01 12:27:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPO', 'BAH', 'BHR', 'DXB', 'ARE', '2021-11-29 00:52:54', '2021-11-29 01:55:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECJ', 'VCE', 'ITA', 'DXB', 'ARE', '2021-11-30 14:08:2


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENE', 'CMB', 'LKA', 'MLE', 'MDV', '2021-11-30 15:17:00', '2021-11-30 16:29:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPE', 'DXB', 'ARE', 'ISB', 'PAK', '2021-11-28 23:37:52', '2021-11-29 02:04:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENK', 'KWI', 'KWT', 'DXB', 'ARE', '2021-11-29 00:28:55', '2021-11-29 01:57:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPO', 'BKK', 'THA', 'HKT', 'THA', '2021-11-30 14:21:00', '2021-11-30 15:30:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EOW', 'DXB', 'ARE', 'VIE', 'AUT', '2021-11-30 05:22:00', '2021-11-30 11:15:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGW', 'DXB', 'ARE', 'ZAZ', 'ESP', '2021-11-28 05:46:00', '2021-11-28 13:17:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQC', 'DXB', 'ARE', 'AMM', 'JOR', '2021-11-27 18:43:00', '2021-11-27 21:34:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQH', 'RUH', 'SAU', 'DXB', 'ARE', '2021-11-30 06:58:3


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENE', 'DXB', 'ARE', 'ACC', 'GHA', '2021-11-29 04:25:21', '2021-11-29 12:02:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EEJ', 'DXB', 'ARE', 'HAM', 'DEU', '2021-11-28 05:35:00', '2021-11-28 12:08:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPH', 'DXB', 'ARE', 'AMD', 'IND', '2021-11-29 18:57:40', '2021-11-29 21:09:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUB', 'LHR', 'GBR', 'DXB', 'ARE', '2021-11-30 22:29:00', '2021-12-01 04:22:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGC', 'DXB', 'ARE', 'BLR', 'IND', '2021-11-29 18:00:00', '2021-11-29 21:41:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPK', 'DXB', 'ARE', 'BUD', 'HUN', '2021-11-29 06:48:00', '2021-11-29 12:25:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQH', 'DXB', 'ARE', 'BRU', 'BEL', '2021-11-29 04:39:00', '2021-11-29 11:53:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPB', 'ADD', 'ETH', 'DXB', 'ARE', '2021-11-30 12:31:3


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECY', 'DXB', 'ARE', 'BHX', 'GBR', '2021-11-28 04:11:00', '2021-11-28 11:42:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUN', 'MAN', 'GBR', 'DXB', 'ARE', '2021-11-30 21:24:00', '2021-12-01 03:37:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPU', 'LIS', 'PRT', 'DXB', 'ARE', '2021-11-28 13:59:30', '2021-11-28 21:12:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPP', 'DAC', 'BGD', 'DXB', 'ARE', '2021-11-29 04:23:00', '2021-11-29 09:10:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ENS', 'RUH', 'SAU', 'DXB', 'ARE', '2021-11-30 17:08:33', '2021-11-30 18:43:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGT', 'MEL', 'AUS', 'BKK', 'THA', '2021-11-27 23:39:00', '2021-11-28 08:33:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGA', 'SYD', 'AUS', 'DXB', 'ARE', '2021-11-29 11:04:22', '2021-11-30 00:57:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EBM', 'DXB', 'ARE', 'CPH', 'DNK', '2021-11-29 19:39:0


    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EPL', 'DXB', 'ARE', 'IKA', 'IRN', '2021-11-30 21:28:00', '2021-11-30 23:21:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EVB', 'MAN', 'GBR', 'DXB', 'ARE', '2021-11-27 21:08:00', '2021-11-28 03:34:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EQP', 'DXB', 'ARE', 'CDG', 'FRA', '2021-11-29 00:03:00', '2021-11-29 06:54:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-ECK', 'DAR', 'TZA', 'DXB', 'ARE', '2021-11-28 15:38:07', '2021-11-28 20:35:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUR', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-29 17:39:00', '2021-11-29 20:04:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EGQ', 'DXB', 'ARE', 'DMM', 'SAU', '2021-11-30 12:14:00', '2021-11-30 13:18:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EUH', 'LHR', 'GBR', 'DXB', 'ARE', '2021-11-28 09:52:00', '2021-11-28 15:55:00')
    

    INSERT INTO FlightLeg2 VALUES(NULL,'A6-EBK', 'ORD', 'USA', 'DXB', 'ARE', '2021-11-29 18:25:5